In [1]:
# For reading data
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# For visualizing
import plotly.express as px

# For model building
import torch
import torch.nn as nn
import torch.nn.functional as F

import requests
import gzip
import io
import numpy as np

In [2]:
class FashionMNISTIDX(Dataset):
    def __init__(self, image_url, label_url, transform=None):
        # Load labels
        response = requests.get(label_url)
        with gzip.open(io.BytesIO(response.content), 'rb') as lbpath:
            labels = np.frombuffer(lbpath.read(), dtype=np.uint8, offset=8)

        # Load images
        response = requests.get(image_url)
        with gzip.open(io.BytesIO(response.content), 'rb') as imgpath:
            images = np.frombuffer(imgpath.read(), dtype=np.uint8, offset=16).reshape(len(labels), 28, 28)

        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(torch.tensor(image, dtype=torch.float32))

        return image, label

In [3]:
train = FashionMNISTIDX('https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-images-idx3-ubyte.gz',
                          'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-labels-idx1-ubyte.gz')

test = FashionMNISTIDX('https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-images-idx3-ubyte.gz',
                       'https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-labels-idx1-ubyte.gz')

train_dataloader = DataLoader(train, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test, batch_size=64, shuffle=False)

In [12]:
class SimpleFashionNet(nn.Module):
    def __init__(self):
        super(SimpleFashionNet, self).__init__()
        # Flatten the input image
        self.flatten = nn.Flatten()
        # Fully connected layer that maps the 28x28 input to 10 output classes
        self.fc = nn.Linear(28*28, 10)

    def forward(self, x):
        # Flatten the input image
        x = self.flatten(x)
        # Apply the fully connected layer
        x = self.fc(x)
        # Apply log softmax to the output
        return F.log_softmax(x, dim=1)

# Instantiate the model
model = SimpleFashionNet()

In [5]:
# Define some training parameters
learning_rate = 1e-2
batch_size = 64
epochs = 30

# Define our loss function
#   This one works for multiclass problems
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)  # Get the total number of samples in the dataset
    model.train()  # Set the model to training mode
    for batch, (X, y) in enumerate(dataloader):  # Iterate over batches
        X, y = X.float(), y  # Convert inputs to float
        pred = model(X)  # Get model predictions
        loss = loss_fn(pred, y)  # Calculate the loss
        optimizer.zero_grad()  # Zero the gradients
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update the model parameters

        if batch % 100 == 0:  # Print loss every 100 batches
            loss, current = loss.item(), batch * len(X)  # Get the current loss and number of samples processed
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")  # Print the loss and progress

In [ ]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)  # Total number of samples in the dataset
    num_batches = len(dataloader)  # Total number of batches
    test_loss, correct = 0, 0  # Initialize test loss and correct predictions count
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation
        for X, y in dataloader:  # Iterate over batches
            X, y = X.float(), y  # Convert inputs to float
            pred = model(X)  # Get model predictions
            test_loss += loss_fn(pred, y).item()  # Accumulate loss
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()  # Count correct predictions

    test_loss /= num_batches  # Calculate average loss
    correct /= size  # Calculate accuracy
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")  # Print results

In [8]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 118.771233 [    0/60000]
loss: 1409.493896 [ 6400/60000]
loss: 4240.417969 [12800/60000]


c:\Users\Ghaith\anaconda3\Lib\site-packages\torch\utils\data\_utils\collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:212.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


loss: 3014.708496 [19200/60000]
loss: 5032.420898 [25600/60000]
loss: 694.342834 [32000/60000]
loss: 281.754639 [38400/60000]
loss: 1292.497070 [44800/60000]
loss: 800.562500 [51200/60000]
loss: 534.525146 [57600/60000]
Test Error: 
 Accuracy: 79.5%, Avg loss: 988.187885 

Epoch 2
-------------------------------
loss: 571.791992 [    0/60000]
loss: 2424.322998 [ 6400/60000]
loss: 1273.923828 [12800/60000]
loss: 459.013855 [19200/60000]
loss: 1364.981812 [25600/60000]
loss: 753.615479 [32000/60000]
loss: 1255.692505 [38400/60000]
loss: 868.303284 [44800/60000]
loss: 695.403076 [51200/60000]
loss: 617.057617 [57600/60000]
Test Error: 
 Accuracy: 78.9%, Avg loss: 973.054800 

Epoch 3
-------------------------------
loss: 858.107239 [    0/60000]
loss: 767.272827 [ 6400/60000]
loss: 2632.609131 [12800/60000]
loss: 1228.200317 [19200/60000]
loss: 604.835999 [25600/60000]
loss: 902.173218 [32000/60000]
loss: 630.164001 [38400/60000]
loss: 590.805115 [44800/60000]
loss: 5214.740723 [51200/600

In [9]:
model.eval()

image, truth = test_dataloader.dataset.__getitem__(1)
image = torch.tensor(image).float().unsqueeze(0)

pred = model(image).argmax()

print(f"This image is predicted to be a {pred.item()}, and is labeled as {truth}")

This image is predicted to be a 4, and is labeled as 2


In [ ]:
# Save our model
EPOCH = epochs
PATH = "model.pt"

# The save function creates a binary storing all our data for us
torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, PATH)

In [11]:
PATH = "model.pt"

# Create a new "blank" model to load our information into
model = SimpleFashionNet()

# Recreate our optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Load back all of our data from the file
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
EPOCH = checkpoint['epoch']

C:\Users\Ghaith\AppData\Local\Temp\ipykernel_25708\2088115640.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH)
